## Beregne
<font size=2>Veilederen har forløpig eksempler for funksjonene <b>avlede variabler</b> og <b>fordelingsberegninger</b></font>

<font size=2><b>Avlede variabler:</b> <code>dataframe<b>.withColumn</b>('avledet variabel','avledningsbetingelse|avledning')</code>

Eksempel: Summere variabelene x og y fra et datasett df til en ny variabel y: <code>df.withcolumn('z',df['x'] + df['y'])</code>

I eksemplet under lager vi en ny variabel <b>Befolkningstetthet</b> fra variablene <i>innbyggerantall</i> og <i>areal</i>,  men først må vi koble datasettene df_innbyggerantall og df_areal
</font>

#### Importere biblioteker (kode)

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

#### Hente eksempeldata (kode)

In [ ]:
df_areal = spark.read.path("/felles/veiledning/pyspark/eksempler/areal")
df_bnp = spark.read.path("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")

#### Koble datasettene med areal og innbyggerantall sammen (kode)

In [ ]:
#Først fjerner vi de kolonnene vi ikke trenger for eksempelet
df_areal=df_areal.drop('Årstall', 'Landkode', 'Kilde')
df_innbyggerantall = df_innbyggerantall.drop('År','Landkode','Kilde')

#Så kobles datasettene sammen med koblingsnøkkelen 'outer' (dvs. union)
df_areal_innbyggerantall = df_areal.join(df_innbyggerantall, 'Land','outer')

#### Avlede variabelen befolkningstetthet med withColumn (kode)

In [ ]:
df_areal_innbyggerantall = df_areal_innbyggerantall.withColumn('Befolkningstetthet',df_areal_innbyggerantall['Innbyggerantall']/df_areal_innbyggerantall['Areal'])
df_areal_innbyggerantall.show()

### Fordelingsberegninger
<font size=2>
<ul>
<li>Snitt    : <code>dataframe.<b>groupBy</b>("grupperingsvariabel").<b>agg</b>(<b>mean</b>('variabel man skal beregne gjennomsnittet av'))</code>
<li>Antall   : <code>dataframe.<b>groupBy</b>("grupperingsvariabel").<b>count()</b></code>
<li>Minimum  : <code>dataframe.<b>groupBy</b>("grupperingsvariabel").<b>agg</b>(<b>min</b>('variabel man skal beregne minste verdi av'))</code>
<li>Maksimum : <code>dataframe.<b>groupBy</b>("grupperingsvariabel").<b>agg</b>(<b>max</b>('variabel man skal beregne maksverdi av'))</code>
</ul>

Nødvendige biblioteker:
<code>from pyspark.sql.functions import mean, min, max</code>
</font>

### Lage en grupperingsvariabel
<font size=2>For å vise eksempler på fordelingsberegninger lager vi en grupperingsvariabel på befolkningstettet: <b>Tetthetsgruppe</b> hvor befolkningstettet under 100 er gruppe 1 mens de over er i gruppe 2. Dette gjøres ved å benytte en logisk betingelse med funksjonene <code>when('betingelse', 'sannverdi').otherwise('usannverdi')</code> i kombinasjon med <code>withColumn</code>. For å kunne sette en konstantverdi (1 eller 2) må vi benytte funksjonen <code>lit('konstantverdi')</code>.

Funksjonene <code>when</code> og <code>lit()</code> krever import av bibliotek:
<code>import pyspark.sql.functions as F</code>

</font>

#### Avlede variabelen tetthetsgruppe med withColumn (kode)

In [ ]:
df_areal_innbyggerantall = df_areal_innbyggerantall.withColumn('Tetthetsgruppe', F.when(df_areal_innbyggerantall.Befolkningstetthet<100,F.lit(1)).otherwise(F.lit(2)))
df_areal_innbyggerantall.show()

#### Telle opp antall land i de ulike gruppene (kode)

In [ ]:
tetthetsgruppe_count = df_areal_innbyggerantall.groupBy(df_areal_innbyggerantall.Tetthetsgruppe).count()
tetthetsgruppe_count.show()

#### Gjennomsnitt og maks innbyggerantall pr gruppe (kode)

In [ ]:
tetthetsgruppe_mean_max = df_areal_innbyggerantall.\
                            groupBy(df_areal_innbyggerantall.Tetthetsgruppe).agg(\
                            F.mean(df_areal_innbyggerantall.Innbyggerantall).cast(LongType()),\
                            F.max(df_areal_innbyggerantall.Innbyggerantall))
tetthetsgruppe_mean_max.printSchema()

tetthetsgruppe_mean_max.show()

### Summere en kolonne og telle opp antall forekomster
<font size=2>Sum: <code><i>dataframe</i>.select().F.sum(<i>dataframe.kolonnenavn</i>).collect()[0][0]</code>
Antall: <code><i>dataframe</i>.select().F.count(<i>dataframe.kolonnenavn</i>).collect()[0][0]</code>

Grunnen til at det står [0][0] er at spørringen gir en dataframe tilbake og ikke et tall. Så [0][0] viser til den første raden i den første kolonnen i dataframen. I dette tilfellet er det den eneste elementen i dataframen så da må det være [0][0]. Ved å gjøre dette får vi et tall tilbake, og ikke en dataframe
</font>

#### Summere og telle opp (kode)

In [ ]:
sum_antall_innbygger = df_areal_innbyggerantall.select(F.sum(df_areal_innbyggerantall.Innbyggerantall)).collect()[0][0]

print("antall innbyggere : "+ repr(sum_antall_innbygger))

antall_land = df_areal_innbyggerantall.select(F.count(df_areal_innbyggerantall.Innbyggerantall)).collect()[0][0]


print("antall land : "+ repr(antall_land))